In [1]:
# https://arxiv.org/abs/2101.00027    -    BIG DATA-SET
# https://huggingface.co/datasets/enwik8 - SMALLER FOR TESTING?

In [2]:
import numpy as np
import torch
from tqdm import tqdm
import torchtext
import numpy as np
import torch.optim as optim
import torch.utils.data as data
from torchtext.data import get_tokenizer

from tokenizers import Tokenizer # RWKV https://github.com/BlinkDL/ChatRWKV/blob/main/RWKV_in_150_lines.py#L11
tokenizer = Tokenizer.from_file("20B_tokenizer.json") # RWKV https://github.com/BlinkDL/ChatRWKV/blob/main/RWKV_in_150_lines.py#L11

In [3]:
folderr = "simplebooks/simplebooks-92"
train_file = folderr + "/train.txt"
test_file = folderr + "/test.txt"
validation_file = folderr + "/validation.txt"

In [4]:
filename = train_file
raw_text = open(filename, 'r', encoding='utf-8').read()

#tokenizer = get_tokenizer("subword")

#temp = tokenizer.encode(raw_text)
raw_text = tokenizer.encode(raw_text).ids
n_vocab = 50276 + 1 # TOKENIZER 20B_tokenizer.json
#n_vocab = max(raw_text) + 1 # 50276 + 1 for 20B_tokenizer - GENERAL

print(n_vocab)

50277


In [5]:
#chars = sorted(list(set(raw_text)))
#char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:
n_tokens = len(raw_text)
#n_vocab = len(chars)
print("Total Tokens: ", n_tokens)

Total Tokens:  3809999


In [7]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_tokens - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append(seq_in) # [char_to_int[char] for char in seq_in]
    dataY.append(seq_out)
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  3809899


In [8]:
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)

In [9]:
lookback = 1
print(X.shape, y.shape)

torch.Size([3809899, 100, 1]) torch.Size([3809899])


In [10]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=100, num_layers=1, batch_first=True)
        self.linear1 = nn.Linear(100, 50)
        self.linear2 = nn.Linear(50, 25)
        self.linear3 = nn.Linear(25, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        return x

In [11]:
device = torch.device("cuda:0") # cpu - cpu / cuda:0 - gpu

model = LSTMModel().to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=128)

best_model = None
best_loss = np.inf

In [12]:
current_epoch = 0

In [13]:
n_epochs = 10000

In [ ]:
 for e in range(n_epochs):
    current_epoch += 1
    model.float()
    model.train()
    for X_batch, y_batch in tqdm(loader):
        y_pred = model(X_batch.float().to(device))
        loss = loss_fn(y_pred.to(device), y_batch.long().to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.float().to(device))
            loss += loss_fn(y_pred.to(device), y_batch.long().to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
            torch.save(best_model, "fnn-tokenizer.pth")
        print("Epoch %d: Cross-entropy: %.4f" % (current_epoch, loss))

# 2 layers LSTM -> Epoch 13: Cross-entropy: 154954.5625

 18%|█▊        | 5259/29765 [00:44<03:17, 124.04it/s]

In [15]:
seq_length = 100
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]

In [20]:
import numpy as np
import torch
import torch.nn as nn

best_model = torch.load("double-rwkv-tokenizer.pth")

model = LSTMModel()
model.load_state_dict(best_model)

filename = "wonderland.txt"
seq_length = 100
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = 'The way you'
print("xd " + str(prompt) + " dx ")
pattern = tokenizer.encode(prompt).ids

model.eval()
print('Prompt: "%s"' % prompt)

tokens_to_decode = []

with torch.no_grad():
    for i in range(1000):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x)
        # convert logits into one character
        #index = int(prediction.argmax())
        #print(str(tokenizer.decode([prediction.argmax()])))
        index = prediction.argmax()
        tokens_to_decode.append(index)
        result = tokenizer.decode(tokens_to_decode)
        if '\ufffd' not in result:
            print(result, end="", flush=True)
            tokens_to_decode = []
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

xd The way you dx 
Prompt: "The way you"
 thebe of the world , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , and the little , an

In [ ]:
print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())